<a href="https://colab.research.google.com/github/talktokorea/storage/blob/master/lecture5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0. 한글 폰트 다운로드 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 0. 구글 드라이브를 코랩에 마운트 Mount Google Drive
from google.colab import drive     # import drive from google colab
ROOT="/content/drive"                # root directory
drive.mount(ROOT)                     # mount google drive at ROOT= /content/drive
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/storage"
%cd {PROJECT_PATH}
%pwd

In [ ]:
# 1. 농어 Perch 데이터 입력
import numpy as np
perch_length = np.array(
      [8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0, 21.0,
       21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5, 22.5, 22.7,
       23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5, 27.3, 27.5, 27.5,
       27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0, 36.5, 36.0, 37.0, 37.0,
       39.0, 39.0, 39.0, 40.0, 40.0, 40.0, 40.0, 42.0, 43.0, 43.0, 43.5,
       44.0]
       )
perch_weight = np.array(
      [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0,
       115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
       150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
       218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
       556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
       850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
       1000.0]
      )

In [ ]:
# 2. 데이터 산포도
import matplotlib.pyplot as plt
plt.scatter(perch_length,perch_weight)
plt.xlabel('length')
plt.ylabel('weight')      # 예측값 y=무게
plt.show()

In [ ]:
# 3. 훈련 세트와 테스트 세트를 구분(25% 테스트 세트 기본 설정) (각각 입력 및 정답)
# train_test_split() 이용 train_input, train_targt, test_input, test_target 확보
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target = train_test_split( perch_length, perch_weight, random_state=42)
print(train_input.shape, test_input.shape)
print(train_target.shape, test_target.shape)
print(test_target)

In [ ]:
# 4. 훈련세트 + 테스트세트 산포도 확인
import matplotlib.pyplot as plt
plt.scatter(train_input,train_target)
plt.scatter(test_input,test_target)
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
# 5. 사이킷런에서 사용할 수 있도록 2차원 배열 데이터로 변경
train_input=train_input.reshape(-1,1)
#train_input = np.reshape(train_input,(-1,1)) # 동일
#test_input=test_input.reshape(-1,1)
test_input = np.reshape(test_input,(-1,1)) # 동일
#print(train_input)
print(test_input)

In [ ]:
# 6. 훈련 및 결정계수(테스트) 평가
from sklearn.neighbors import KNeighborsRegressor  # 사이킷런 패키지
knr = KNeighborsRegressor()   # 객체 생성
knr.fit(train_input,train_target)    # 머신러닝 훈련
knr.score(test_input, test_target)  # 결정계수 R 평가  R테스트=0.992
test_prediction=knr.predict(test_input)             # 예측

from sklearn.metrics import mean_absolute_error  # 사이킷런 패키지
mae = mean_absolute_error(test_target,test_prediction) #타겟예측절대오차
print(mae)                      # 예측이 평균적으로 19g 정도 타겟값과 다르다는 의미

In [ ]:
# 7. 훈련 데이터 이용한 결정계수(훈련) 추가평가
#from sklearn.neighbors import KNeighborsRegressor  # 사이킷런 패키지
#knr = KNeighborsRegressor()   # 객체 생성
#knr.fit(train_input,train_target)    # 머신러닝 훈련
knr.score(train_input, train_target)  # 결정계수 R 평가  R훈련=0.969

In [ ]:
# 8. R훈련 < R테스트 이므로 과소적합 상태임
#복잡도 높이기 (k값 낮추기) 필요함
knr.n_neighbors = 3    # 5에서 3으로 감소 설정
knr.fit(train_input,train_target)
print(knr.score(train_input,train_target))    # R훈련 = 0.980
print(knr.score(test_input,test_target))      # R테스트 = 0.974

In [ ]:
# 과대 적합 추가 토론
from sklearn.neighbors import KNeighborsRegressor  # 사이킷런 패키지
knr = KNeighborsRegressor()   # 객체 생성
x=np.arange(5,45).reshape(-1,1)   # 5에서 45까지 x 좌표 생성
# k=1,5,10 일때 예측결과를 그래프로 그리기
for k in [1,5,10]:
   # 모델 훈련
   knr.n_neighbors = k                   # 이웃 숫자 설정
   knr.fit(train_input,train_target)    # 머신러닝 훈련
   # 지정한 범위 x에 대한 예측 구하기
   prediction = knr.predict(x)
   # 훈련 세트와 에측 결과 그래프
   plt.scatter(train_input,train_target)
   plt.plot(x,prediction)      # 선형그래프
   plt.title('n_neighbors={}'.format(k))
   plt.xlabel('length')
   plt.ylabel('weight')
   plt.show()